In [ ]:
!pip install flask
!pip install sentence-transformers
!pip install pyngrok

In [ ]:
!pip install flask_cors


In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
from sentence_transformers import SentenceTransformer, util
import json
from pyngrok import ngrok

app = Flask(__name__)

# Enable CORS for the Flask app
CORS(app)  # This allows cross-origin requests from your frontend


# Initialize the Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Choose an appropriate model

# Load questions and answers from a JSON file
with open('/content/questions.json', 'r') as file:
    data = json.load(file)

# Precompute embeddings for all correct answers
for question in data['questions']:
    question['answer_embeddings'] = model.encode(question['answers'], convert_to_tensor=True)

@app.route('/')
def home():
    return "Hello from the AI Interview Prep App on Colab!"

@app.route('/api/evaluate', methods=['POST'])
def evaluate():
    content = request.json
    question_id = content.get('question_id')
    user_response = content.get('user_response')

    if not question_id or not user_response:
        return jsonify({"error": "Missing question_id or user_response"}), 400

    # Find the relevant question
    question = next((q for q in data['questions'] if q['id'] == question_id), None)
    if not question:
        return jsonify({"error": "Question not found"}), 404

    # Encode the user response
    user_embedding = model.encode(user_response, convert_to_tensor=True)

    # Compute cosine similarities with all correct answers
    cosine_scores = util.pytorch_cos_sim(user_embedding, question['answer_embeddings'])

    # Get the maximum similarity score
    max_score = cosine_scores.max().item()

    # Determine if the response is adequate
    threshold = 0.75  # Adjust based on testing and requirements
    is_adequate = max_score >= threshold

    # Generate feedback
    if is_adequate:
        feedback = "Good response! Your answer aligns well with the ideal responses."
    else:
        feedback = "Your response could be improved. Try to elaborate more on key points."

    return jsonify({
        "similarity_score": round(max_score, 2),
        "is_adequate": is_adequate,
        "feedback": feedback
    }), 200


In [ ]:
!ngrok authtoken 2od9knxpTCsFfAJBqAQ50mG61ik_75AUvcZ5q214GUVjaLamK


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Open a tunnel to the Flask server on port 5000
public_url = ngrok.connect(5000)
print("Public URL:", public_url)



Public URL: NgrokTunnel: "https://ed54-35-229-221-78.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 01:37:22] "OPTIONS /api/evaluate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 01:37:23] "POST /api/evaluate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 01:50:33] "OPTIONS /api/evaluate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 01:50:34] "POST /api/evaluate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 01:50:52] "OPTIONS /api/evaluate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 01:50:52] "POST /api/evaluate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 02:01:24] "OPTIONS /api/evaluate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 02:01:24] "POST /api/evaluate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 02:07:39] "OPTIONS /api/evaluate HTTP/1.1" 200 